In [0]:
import datetime as dt
import requests
from pyspark.sql import functions as F
from pyspark.sql.types import *

TODAY = "2025-05-21"
BRONZE_WEATHER_PATH = f"dbfs:/bronze/weather/{TODAY}"


In [0]:
URL = "https://api.weather.gov/gridpoints/SEW/123,67/forecast/hourly"


In [0]:
print("Status Code:", response.status_code)
print("Response Headers:", response.headers)
print("Raw Response:", response.text[:500])  # Show only first 500 characters


In [0]:
props = data.get("properties", {})
periods = props.get("periods", [])

print(f"Updated at: {props.get('updated', 'N/A')}")
print(f"Hours returned: {len(periods)}")


In [0]:
df_weather = spark.createDataFrame(periods)

df_weather = (
    df_weather
    .withColumn("forecast_retrieved_at", F.current_timestamp())
    .withColumn("forecast_time", F.to_timestamp("startTime"))
)

df_weather.select("name", "forecast_time", "temperature", "shortForecast").show(5, truncate=False)


In [0]:
df_weather.write.format("delta").mode("overwrite").save(BRONZE_WEATHER_PATH)
print("✓ Hourly forecast saved to Bronze")
